## Part 1: Language Modeling

In this project, you will implement several different types of language models for text.  We'll start with n-gram models and then move on to neural n-gram.

Warning: Do not start this project the day before it is due!  Some parts require 20 minutes or more to run, so debugging and tuning can take a significant amount of time.

Our dataset for this project will be the Penn Treebank language modeling dataset.  This dataset comes with some of the basic preprocessing done for us, such as tokenization and rare word filtering (using the `<unk>` token).
Therefore, we can assume that all word types in the test set also appear at least once in the training set.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# This block handles some basic setup and data loading.
# You shouldn't need to edit this, but if you want to
# import other standard python packages, that is fine.

# imports
from collections import defaultdict, Counter
import numpy as np
import math
import tqdm
import random
import pdb

import torch
from torch import nn
import torch.nn.functional as F
from datasets import Dataset
import os

# Load WikiText-2 dataset from local arrow files
# The arrow files are in ./wikitext/wikitext-2-v1/
cache_path = "/content/drive/MyDrive/wikitext/wikitext-2-v1"

# Load the three splits from arrow files using Dataset.from_file()
train_dataset = Dataset.from_file(os.path.join(cache_path, "wikitext-train.arrow"))
validation_dataset = Dataset.from_file(os.path.join(cache_path, "wikitext-validation.arrow"))
test_dataset = Dataset.from_file(os.path.join(cache_path, "wikitext-test.arrow"))

# Convert to list of tokens (HuggingFace returns text as strings, we need to tokenize)
# WikiText-2 is already tokenized with space-separated tokens
def get_tokens(example):
    # Split by whitespace and filter out empty strings
    tokens = example['text'].split()
    return tokens

# Get all tokens from each split
train_text = []
for example in train_dataset:
    tokens = get_tokens(example)
    if tokens:  # Skip empty examples
        train_text.extend(tokens)

validation_text = []
for example in validation_dataset:
    tokens = get_tokens(example)
    if tokens:
        validation_text.extend(tokens)

test_text = []
for example in test_dataset:
    tokens = get_tokens(example)
    if tokens:
        test_text.extend(tokens)

# Build vocabulary from training set only
# (Validation and test sets may contain unknown tokens, which will be mapped to <unk>)
token_counts = Counter(train_text)

# Create vocabulary with special tokens
# Add special tokens: <unk> for unknown words, <eos> for end of sentence
special_tokens = ['<unk>', '<eos>', '<pad>']
for token in special_tokens:
    token_counts[token] = 0
vocab_list = sorted(token_counts.keys())
vocab_size = len(vocab_list)

# Create a simple vocab class compatible with torchtext interface
class Vocab:
    def __init__(self, vocab_list, token_counts):
        self.itos = vocab_list  # index to string
        self.stoi = {word: idx for idx, word in enumerate(vocab_list)}  # string to index
        self.freqs = token_counts  # frequency counts

    def __len__(self):
        return len(self.itos)

vocab = Vocab(vocab_list, token_counts)

print(f"Vocabulary size: {vocab_size}")
print(f"First 30 validation tokens: {validation_text[:30]}")

Vocabulary size: 33279
First 30 validation tokens: ['=', 'Homarus', 'gammarus', '=', 'Homarus', 'gammarus', ',', 'known', 'as', 'the', 'European', 'lobster', 'or', 'common', 'lobster', ',', 'is', 'a', 'species', 'of', '<unk>', 'lobster', 'from', 'the', 'eastern', 'Atlantic', 'Ocean', ',', 'Mediterranean', 'Sea']


In [ ]:
print(validation_text[:300])

['=', 'Homarus', 'gammarus', '=', 'Homarus', 'gammarus', ',', 'known', 'as', 'the', 'European', 'lobster', 'or', 'common', 'lobster', ',', 'is', 'a', 'species', 'of', '<unk>', 'lobster', 'from', 'the', 'eastern', 'Atlantic', 'Ocean', ',', 'Mediterranean', 'Sea', 'and', 'parts', 'of', 'the', 'Black', 'Sea', '.', 'It', 'is', 'closely', 'related', 'to', 'the', 'American', 'lobster', ',', 'H.', 'americanus', '.', 'It', 'may', 'grow', 'to', 'a', 'length', 'of', '60', 'cm', '(', '24', 'in', ')', 'and', 'a', 'mass', 'of', '6', 'kilograms', '(', '13', 'lb', ')', ',', 'and', 'bears', 'a', 'conspicuous', 'pair', 'of', 'claws', '.', 'In', 'life', ',', 'the', 'lobsters', 'are', 'blue', ',', 'only', 'becoming', '"', 'lobster', 'red', '"', 'on', 'cooking', '.', 'Mating', 'occurs', 'in', 'the', 'summer', ',', 'producing', 'eggs', 'which', 'are', 'carried', 'by', 'the', 'females', 'for', 'up', 'to', 'a', 'year', 'before', 'hatching', 'into', '<unk>', 'larvae', '.', 'Homarus', 'gammarus', 'is', 'a', 'h

We've implemented a unigram model here as a demonstration.

In [ ]:
class UnigramModel:
    def __init__(self, train_text):
        self.counts = Counter(train_text)
        self.total_count = len(train_text)

    def probability(self, word):
        # print("total counts:", self.total_count)
        # print(f"self.counts[{word}]", self.counts[word])

        return self.counts[word] / self.total_count

    def next_word_probabilities(self, text_prefix):
        """Return a list of probabilities for each word in the vocabulary."""
        return [self.probability(word) for word in vocab.itos]

    def perplexity(self, full_text):
        """Return the perplexity of the model on a text as a float.

        full_text -- a list of string tokens
        """
        log_probabilities = []
        for word in full_text:
            # Note that the base of the log doesn't matter
            # as long as the log and exp use the same base.
            prob = self.probability(word)
            # print(f"prob for {word}: {prob}")
            # Handle 0 probability by using a very small epsilon to avoid log(0)
            if prob == 0:
                prob = 1e-10
            log_probabilities.append(math.log(prob, 2))
        return 2 ** -np.mean(log_probabilities)

unigram_demonstration_model = UnigramModel(train_text)
print('unigram validation perplexity:',
      unigram_demonstration_model.perplexity(validation_text))

def check_validity(model):
    """Performs several sanity checks on your model:
    1) That next_word_probabilities returns a valid distribution
    2) That perplexity matches a perplexity calculated from next_word_probabilities

    Although it is possible to calculate perplexity from next_word_probabilities,
    it is still good to have a separate more efficient method that only computes
    the probabilities of observed words.
    """

    log_probabilities = []
    for i in range(10):
        prefix = validation_text[:i]
        probs = model.next_word_probabilities(prefix)
        print("sum of probs:", abs(sum(probs)-1))
        assert min(probs) >= 0, "Negative value in next_word_probabilities"
        assert max(probs) <= 1 + 1e-8, "Value larger than 1 in next_word_probabilities"
        assert abs(sum(probs)-1) < 1e-4, "next_word_probabilities do not sum to 1"

        word_id = vocab.stoi[validation_text[i]]
        selected_prob = probs[word_id]
        # Handle 0 probability by using a very small epsilon to avoid log(0)
        if selected_prob == 0:
            selected_prob = 1e-10
        log_probabilities.append(math.log(selected_prob, 2))

    perplexity = 2 ** -np.mean(log_probabilities)
    your_perplexity = model.perplexity(validation_text[:10])
    print("perplexity", perplexity)
    print("your_perplexity", your_perplexity)
    assert abs(perplexity-your_perplexity) < 0.1, "your perplexity does not " + \
    "match the one we calculated from `next_word_probabilities`,\n" + \
    "at least one of `perplexity` or `next_word_probabilities` is incorrect.\n" + \
    f"we calcuated {perplexity} from `next_word_probabilities`,\n" + \
    f"but your perplexity function returned {your_perplexity} (on a small sample)."


check_validity(unigram_demonstration_model)


unigram validation perplexity: 996.5031614073108
sum of probs: 0.0
sum of probs: 0.0
sum of probs: 0.0
sum of probs: 0.0
sum of probs: 0.0
sum of probs: 0.0
sum of probs: 0.0
sum of probs: 0.0
sum of probs: 0.0
sum of probs: 0.0
perplexity 2278.6599542126614
your_perplexity 2278.6599542126614


To generate from a language model, we can sample one word at a time conditioning on the words we have generated so far.

In [ ]:
def generate_text(model, n=20, prefix=('<eos>', '<eos>')):
    prefix = list(prefix)
    for _ in range(n):
        probs = model.next_word_probabilities(prefix)
        word = random.choices(vocab.itos, probs)[0]
        prefix.append(word)
    return ' '.join(prefix)

print(generate_text(unigram_demonstration_model))

<eos> <eos> of felt cast . the good areas area for still name . by associated radio to the , and influx


In fact there are many strategies to get better-sounding samples, such as only sampling from the top-k words or sharpening the distribution with a temperature.  You can read more about sampling from a language model in this recent paper: https://arxiv.org/pdf/1904.09751.pdf.

You will need to submit some outputs from the models you implement for us to grade.  The following function will be used to generate the required output files.

In [ ]:
!wget https://cal-cs288.github.io/sp21/project_files/proj_1/eval_prefixes.txt
!wget https://cal-cs288.github.io/sp21/project_files/proj_1/eval_output_vocab.txt
!wget https://cal-cs288.github.io/sp21/project_files/proj_1/eval_prefixes_short.txt
!wget https://cal-cs288.github.io/sp21/project_files/proj_1/eval_output_vocab_short.txt

def save_truncated_distribution(model, filename, short=True):
    """Generate a file of truncated distributions.

    Probability distributions over the full vocabulary are large,
    so we will truncate the distribution to a smaller vocabulary.

    Please do not edit this function
    """
    vocab_name = 'eval_output_vocab'
    prefixes_name = 'eval_prefixes'

    if short:
      vocab_name += '_short'
      prefixes_name += '_short'

    with open('{}.txt'.format(vocab_name), 'r') as eval_vocab_file:
        eval_vocab = [w.strip() for w in eval_vocab_file]
    # Map unknown words to <unk> token ID
    unk_id = vocab.stoi['<unk>']
    eval_vocab_ids = [vocab.stoi.get(s, unk_id) for s in eval_vocab]

    all_selected_probabilities = []
    with open('{}.txt'.format(prefixes_name), 'r') as eval_prefixes_file:
        lines = eval_prefixes_file.readlines()
        for line in tqdm.tqdm(lines, leave=False):
            prefix = line.strip().split(' ')
            probs = model.next_word_probabilities(prefix)
            selected_probs = np.array([probs[i] for i in eval_vocab_ids], dtype=np.float32)
            all_selected_probabilities.append(selected_probs)

    all_selected_probabilities = np.stack(all_selected_probabilities)
    np.save(filename, all_selected_probabilities)
    print('saved', filename)

--2026-02-09 18:16:17--  https://cal-cs288.github.io/sp21/project_files/proj_1/eval_prefixes.txt
Resolving cal-cs288.github.io (cal-cs288.github.io)... 185.199.108.153, 185.199.109.153, 185.199.110.153, ...
Connecting to cal-cs288.github.io (cal-cs288.github.io)|185.199.108.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 519055 (507K) [text/plain]
Saving to: ‘eval_prefixes.txt’

eval_prefixes.txt   100%[===================>] 506.89K  --.-KB/s    in 0.02s   

2026-02-09 18:16:17 (32.9 MB/s) - ‘eval_prefixes.txt’ saved [519055/519055]

--2026-02-09 18:16:17--  https://cal-cs288.github.io/sp21/project_files/proj_1/eval_output_vocab.txt
Resolving cal-cs288.github.io (cal-cs288.github.io)... 185.199.108.153, 185.199.109.153, 185.199.110.153, ...
Connecting to cal-cs288.github.io (cal-cs288.github.io)|185.199.108.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12497 (12K) [text/plain]
Saving to: ‘eval_output_vocab.txt’

eval_output_

In [ ]:
save_truncated_distribution(unigram_demonstration_model, 'unigram_predictions.npy')

saved unigram_predictions.npy


### N-gram Model

Now it's time to implement an n-gram language model.

Because not every n-gram will have been observed in training, use add-alpha smoothing to make sure no output word has probability 0.

$$P(w_2|w_1)=\frac{C(w_1,w_2)+\alpha}{C(w_1)+N\alpha}$$

where $N$ is the vocab size and $C$ is the count for the given bigram.  An alpha value around `3e-3`  should work.

One edge case you will need to handle is at the beginning of the text where you don't have `n-1` prior words.  You can handle this however you like as long as you produce a valid probability distribution, but just using a uniform distribution over the vocabulary is reasonable for the purposes of this project.

A properly implemented bi-gram model should get a perplexity below 550 on the validation set.

**Note**: Do not change the signature of the `next_word_probabilities` and `perplexity` functions.  We will use these as a common interface for all of the different model types.  Make sure these two functions call `n_gram_probability`, because later we are going to override `n_gram_probability` in a subclass.
Also, we suggest pre-computing and caching the counts $C$ when you initialize `NGramModel` for efficiency.

**Deliverable**: Submit the bigram distribution from the Ngram model.

In [ ]:
print(len(vocab_list), len(train_text), train_text[0:30])

33279 2051910 ['=', 'Valkyria', 'Chronicles', 'III', '=', 'Senjō', 'no', 'Valkyria', '3', ':', '<unk>', 'Chronicles', '(', 'Japanese', ':', '戦場のヴァルキュリア3', ',', 'lit', '.', 'Valkyria', 'of', 'the', 'Battlefield', '3', ')', ',', 'commonly', 'referred', 'to', 'as']


In [ ]:
print(['i','am','kamal', 'how', 'are', 'you'][2:-1])
print(vocab.itos[2])
vocab_index = {vocab.itos[i]: i for i,w in enumerate(vocab.itos)}
print(vocab_index['<eos>'], vocab_index['<eos>'],vocab_index['the'])

['kamal', 'how', 'are']
#
1417 1417 31354


In [ ]:
from collections import defaultdict, Counter

class NGramModel:
    def __init__(self, train_text, n=2, alpha=3e-3):
        # get counts and perform any other setup
        self.n = n
        self.smoothing = alpha
        self.vocab_len = len(vocab.itos)
        self.vocab_index = {vocab.itos[i]: i for i,w in enumerate(vocab.itos)}
        self.total_counts = len(train_text)
        self.counts = Counter(train_text)
        self.prefix_probs = self.cache_counts(train_text)


        # YOUR CODE HERE


    def n_gram_probability(self, n_gram):
        """Return the probability of the last word in an n-gram.

        n_gram -- a list of string tokens
        returns the conditional probability of the last token given the rest.
        """
        assert len(n_gram) == self.n

        if self.n == 1:
            return self.counts[n_gram[0]]/self.total_counts

        prob = 1
        prefix_words = n_gram[:-1]
        prefix_words_i = tuple([self.vocab_index.get(word, "<unk>") for word in prefix_words])
        word_i = self.vocab_index[n_gram[-1]]
        if prefix_words_i in self.prefix_counts:
            prefix_word_count =  self.prefix_counts[prefix_words_i].get(word_i, 0)
            prefix_total_count = self.prefix_total[prefix_words_i]
            return (prefix_word_count + self.smoothing)/ (prefix_total_count + self.smoothing*self.vocab_len)
        else:
            return 1.0/self.vocab_len
        # return prob

        # YOUR CODE HERE


    def next_word_probabilities(self, text_prefix):
        """Return a list of probabilities for each word in the vocabulary."""

        if self.n == 1:
            return [self.n_gram_probability([word]) for word in vocab.itos]

        while len(text_prefix) < self.n -1 :
            text_prefix = ['<eos>'] + text_prefix
        return [self.n_gram_probability(text_prefix[-self.n+1:] + [word]) for word in vocab.itos]

        # YOUR CODE HERE
        # use your function n_gram_probability
        # vocab.itos contains a list of words to return probabilities for


    def perplexity(self, full_text):
        """ full_text is a list of string tokens
        return perplexity as a float """

        log_probabilities = []
        for i in range(len(full_text)):
            if i < self.n-1:
                ngram = (['<eos>'] * (self.n-1-i)) + full_text[:i+1]
            else:
                ngram = full_text[i-self.n+1:i+1]
            prob = self.n_gram_probability(ngram)
            # print(f"prob for {ngram}: {prob}")
            # # Handle 0 probability by using a very small epsilon to avoid log(0)
            if prob == 0:
                prob = 1e-10
            log_probabilities.append(math.log(prob, 2))
        return 2 ** -np.mean(log_probabilities)

        # YOUR CODE HERE
        # use your function n_gram_probability
        # This method should differ a bit from the example unigram model because
        # the first n-1 words of full_text must be handled as a special case.

    def cache_counts(self, train_text):
        # convert training text to vocab indecies once.
        train_text_i = [self.vocab_index[word] for word in train_text]
        # we count first
        prefix_counts = defaultdict(Counter)
        prefix_total = Counter()
        for i in range(len(train_text_i) - self.n + 1):
            if i % 100000 == 0: print(f"processed {i} training tokens")
            prefix_words = tuple(train_text_i[i:i+self.n-1])
            # prefix_words_i = tuple([self.vocab_index[word] for word in prefix_words])
            next_word = train_text_i[i+self.n-1]
            prefix_counts[prefix_words][next_word] += 1
            prefix_total[prefix_words] += 1

        self.prefix_counts = prefix_counts
        self.prefix_total = prefix_total



unigram_model = NGramModel(train_text, 1)
check_validity(unigram_model)
print('unigram validation perplexity:', unigram_model.perplexity(validation_text)) # this should be the almost the same as our unigram model perplexity above

bigram_model = NGramModel(train_text, n=2)
check_validity(bigram_model)
print('bigram validation perplexity:', bigram_model.perplexity(validation_text))

trigram_model = NGramModel(train_text, n=3)
check_validity(trigram_model)
print('trigram validation perplexity:', trigram_model.perplexity(validation_text)) # this won't do very well...

save_truncated_distribution(bigram_model, 'bigram_predictions.npy') # this might take a few minutes


processed 0 training tokens
processed 100000 training tokens
processed 200000 training tokens
processed 300000 training tokens
processed 400000 training tokens
processed 500000 training tokens
processed 600000 training tokens
processed 700000 training tokens
processed 800000 training tokens
processed 900000 training tokens
processed 1000000 training tokens
processed 1100000 training tokens
processed 1200000 training tokens
processed 1300000 training tokens
processed 1400000 training tokens
processed 1500000 training tokens
processed 1600000 training tokens
processed 1700000 training tokens
processed 1800000 training tokens
processed 1900000 training tokens
processed 2000000 training tokens
sum of probs: 0.0
sum of probs: 0.0
sum of probs: 0.0
sum of probs: 0.0
sum of probs: 0.0
sum of probs: 0.0
sum of probs: 0.0
sum of probs: 0.0
sum of probs: 0.0
sum of probs: 0.0
perplexity 2278.6599542126614
your_perplexity 2278.6599542126614
unigram validation perplexity: 996.5031614073108
process

saved bigram_predictions.npy


Please download `bigram_predictions.npy` once you finish this section so that you can submit it.

We can also generate samples from the model to get an idea of how it is doing.

In [ ]:
print(generate_text(bigram_model))

<eos> <eos> nickname Nanyang Movies , Fish jack loyalties practices cascading Reviewers overabundance reindeer homosocial bonding decathlon Immortal dwelt . Contemporary Riga


We now free up some RAM, **it is important to run the cell below, otherwise you may quite possibly run out of RAM in the runtime.**

In [ ]:
# Free up some RAM.
del bigram_model
del trigram_model

### Neural N-gram Model

In this section, you will implement a neural version of an n-gram model.  The model will use a simple feedforward neural network that takes the previous `n-1` words and outputs a distribution over the next word.

You will use PyTorch to implement the model.  We've provided a little bit of code to help with the data loading using PyTorch's data loaders (https://pytorch.org/docs/stable/data.html)

A model with the following architecture and hyperparameters should reach a validation perplexity below 230.
* embed the words with dimension 128, then flatten into a single embedding for $n-1$ words (with size $(n-1)*128$)
* run 2 hidden layers with 1024 hidden units, then project down to size 128 before the final layer (ie. 4 layers total).
* use weight tying for the embedding and final linear layer (this made a very large difference in our experiments); you can do this by creating the output layer with `nn.Linear`, then using `F.embedding` with the linear layer's `.weight` to embed the input
* rectified linear activation (ReLU) and dropout 0.1 after first 2 hidden layers. **Note: You will likely find a performance drop if you add a nonlinear activation function after the dimension reduction layer.**
* train for 10 epochs with the Adam optimizer (should take around 15-20 minutes)


We encourage you to try other architectures and hyperparameters, and you will likely find some that work better than the ones listed above.  A proper implementation with these should be enough to receive full credit on the assignment, though.

In [ ]:
def ids(tokens):
    return [vocab.stoi[t] for t in tokens]

assert torch.cuda.is_available(), "no GPU found; in Colab go to 'Edit->Notebook settings' and choose a GPU hardware accelerator; \n in Kaggle go to 'Settings->Accelerator' and choose a GPU hardware accelerator"

class NeuralNgramDataset(torch.utils.data.Dataset):
    def __init__(self, text_token_ids, n):
        self.text_token_ids = text_token_ids
        self.n = n

    def __len__(self):
        return len(self.text_token_ids)

    def __getitem__(self, i):
        if i < self.n-1:
            prev_token_ids = [vocab.stoi['<eos>']] * (self.n-i-1) + self.text_token_ids[:i]
        else:
            prev_token_ids = self.text_token_ids[i-self.n+1:i]

        assert len(prev_token_ids) == self.n-1

        x = torch.tensor(prev_token_ids)
        y = torch.tensor(self.text_token_ids[i])
        return x, y

class NeuralNGramNetwork(nn.Module):
    # a PyTorch Module that holds the neural network for your model

    def __init__(self, n):
        super().__init__()
        self.n = n
        self.output_layer = nn.Linear(128, len(vocab.itos))
        self.fc1 = nn.Linear(128 * (self.n-1), 1024)
        self.fc2 = nn.Linear(1024,1024)
        self.fc3 = nn.Linear(1024,128)
        self.dropout = nn.Dropout(.1)
        # YOUR CODE HERE


    def forward(self, x):
        # x is a tensor of inputs with shape (batch, n-1)
        # this function returns a tensor of log probabilities with shape (batch, vocab_size)
        embedding = self.output_layer.weight

        x_embedding = F.embedding(x, embedding)
        # print(f"x_embedding:", x_embedding.shape)

        # shape_x1 = x_embedding.shape[-1]
        # shape_x2 = x_embedding.shape[-2]
        # print(f"{shape_x1=}, {shape_x2=}")
        # x = x_embedding.reshape(shape_x1 * shape_x2)
        x = x_embedding.reshape(x_embedding.size(0), -1)

        # print(f"x before fc1:", x.shape)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)

        # print(f"x before fc2:", x.shape)
        x = F.relu(self.fc2(x))
        x = self.dropout(x)

        # print(f"x before fc3:", x.shape)
        x = self.fc3(x)

        # print(f"x before output layer:", x.shape)
        x = self.output_layer(x)
        x = F.log_softmax(x, dim=-1)
        # print("sum of x:", torch.sum(x))
        return x

        # YOUR CODE HERE


class NeuralNGramModel:
    # a class that wraps NeuralNGramNetwork to handle training and evaluation
    # it's ok if this doesn't work for unigram modeling
    def __init__(self, n):
        self.n = n
        self.network = NeuralNGramNetwork(n).cuda()

    def train(self):
        dataset = NeuralNgramDataset(ids(train_text), self.n)
        train_loader = torch.utils.data.DataLoader(dataset, batch_size=128, shuffle=True)
        # iterating over train_loader with a for loop will return a 2-tuple of batched tensors
        # the first tensor will be previous token ids with size (batch, n-1),
        # and the second will be the current token id with size (batch, )
        # you will need to move these tensors to GPU, e.g. by using the Tensor.cuda() function.

        # this will take some time to run; use tqdm.tqdm_notebook to get a progress bar
        # (see Project 1a for example)

        criterion = nn.CrossEntropyLoss()
        optimizer = torch.optim.Adam(self.network.parameters())
        running_loss = 0.0
        for epoch in range(10):  # loop over the dataset multiple times
          progress_bar = tqdm.tqdm_notebook(train_loader, desc=f'Epoch {epoch+1}/10')
          progress_bar_perplexity = tqdm.tqdm_notebook(train_loader, desc=f'Perplexity {epoch+1}/10')
          for i, data in enumerate(train_loader, 0):
              inputs, labels = data
              inputs_in_cuda = inputs.cuda()
              labels_in_cuda = torch.tensor(labels).cuda()

              # zero the parameter gradients
              optimizer.zero_grad()

              # forward + backward + optimize
              outputs = self.network(inputs_in_cuda)
              loss = criterion(outputs, labels_in_cuda)
              loss.backward()
              optimizer.step()

              # print statistics
              running_loss += loss.item()
              if i % 200 == 199:    # print every 2000 mini-batches
                  progress_bar.set_postfix({'loss': f'{running_loss / 2000:.3f}'})
                  # print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
                  perplexity = self.perplexity(validation_text[:1000])
                  progress_bar_perplexity.set_postfix({'perplexity': f'{perplexity}'})
                  running_loss = 0.0

        # YOUR CODE HERE




    def next_word_probabilities(self, text_prefix):
        # YOUR CODE HERE
        # don't forget self.network.eval()
        # you will need to convert text_prefix from strings to numbers with the `ids` function
        # if your `perplexity` function below is based on a NeuralNgramDataset DataLoader, you will need to use the same strategy for prefixes with less than n-1 tokens to pass the validity check
        #   the data loader appends extra "<eos>" (end of sentence) tokens to the start of the input so there are always enough to run the network
        self.network.eval()
        # print(f"text_prefix: {text_prefix}")
        if len(text_prefix) < self.n-1:
            text_prefix = (['<eos>'] * (self.n-len(text_prefix)-1)) + text_prefix
        text_ids = torch.tensor([ids(text_prefix[-self.n+1:])]).cuda()
        with torch.no_grad():
          log_probs = self.network(text_ids)
          probs = log_probs.exp()
        results = self.network(text_ids)
        return probs.squeeze(0).cpu().tolist()


    def perplexity(self, text):
        # you may want to use a DataLoader here with a NeuralNgramDataset
        # don't forget self.network.eval()
        # From GSI: I should target 230 for perplexity
        self.network.eval()
        dataset = NeuralNgramDataset(ids(text), self.n)
        test_loader = torch.utils.data.DataLoader(dataset, batch_size=128, shuffle=True)
        log_probabilities = []
        criterion = nn.CrossEntropyLoss()
        for i, data in enumerate(test_loader, 0):
            inputs, labels = data
            log_probs = self.network(inputs.cuda())
            probs = log_probs.exp()
            for y_hat, y in zip(probs, labels):
              prob_for_y = y_hat[y]
              if prob_for_y == 0:
                prob_for_y = 1e-10

              log_probabilities.append(math.log(prob_for_y, 2))
        return 2 ** -torch.mean(torch.tensor(log_probabilities))

        # YOUR CODE HERE


neural_trigram_model = NeuralNGramModel(3)
check_validity(neural_trigram_model)
neural_trigram_model.train()
print('neural trigram validation perplexity:', neural_trigram_model.perplexity(validation_text))

save_truncated_distribution(neural_trigram_model, 'neural_trigram_predictions.npy', short=False)

sum of probs: 2.963715814985335e-07
sum of probs: 1.3837598089594394e-07
sum of probs: 9.371433407068253e-09
sum of probs: 4.458815965335816e-07
sum of probs: 2.5823828764259815e-07
sum of probs: 9.371433407068253e-09
sum of probs: 4.458815965335816e-07
sum of probs: 4.622233973350376e-07
sum of probs: 2.94270648737438e-07
sum of probs: 7.380003808066249e-08
perplexity 33742.8324570219
your_perplexity tensor(33742.8125)


/tmp/ipython-input-1975074855.py:96: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  progress_bar = tqdm.tqdm_notebook(train_loader, desc=f'Epoch {epoch+1}/10')


Epoch 1/10:   0%|          | 0/16031 [00:00<?, ?it/s]

/tmp/ipython-input-1975074855.py:97: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  progress_bar_perplexity = tqdm.tqdm_notebook(train_loader, desc=f'Perplexity {epoch+1}/10')


Perplexity 1/10:   0%|          | 0/16031 [00:00<?, ?it/s]

...

Epoch 10/10:   0%|          | 0/16031 [00:00<?, ?it/s]

Perplexity 10/10:   0%|          | 0/16031 [00:00<?, ?it/s]

neural trigram validation perplexity: tensor(288.1129)















  0%|          | 0/5000 [00:00<?, ?it/s]












  1%|          | 52/5000 [00:00<00:09, 513.40it/s]












  2%|▏         | 108/5000 [00:00<00:09, 538.71it/s]












  3%|▎         | 163/5000 [00:00<00:08, 542.58it/s]












  4%|▍         | 220/5000 [00:00<00:08, 552.19it/s]












  6%|▌         | 276/5000 [00:00<00:08, 551.15it/s]












  7%|▋         | 333/5000 [00:00<00:08, 556.96it/s]












  8%|▊         | 390/5000 [00:00<00:08, 559.84it/s]












  9%|▉         | 447/5000 [00:00<00:08, 561.84it/s]












 10%|█         | 505/5000 [00:00<00:07, 565.03it/s]












 11%|█         | 562/5000 [00:01<00:07, 562.81it/s]












 12%|█▏        | 619/5000 [00:01<00:08, 540.80it/s]












 14%|█▎        | 676/5000 [00:01<00:07, 548.57it/s]












 15%|█▍        | 732/5000 [00:01<00:07, 549.28it/s]












 16%|█▌        | 789/5000 [00:01<00:07, 553.77it/s]












 17%|█▋        | 845/5000 [00:01<00:07,

saved neural_trigram_predictions.npy


In [ ]:

save_truncated_distribution(neural_trigram_model, 'neural_trigram_predictions.npy', short=False)














  0%|          | 0/5000 [00:00<?, ?it/s]












  1%|          | 55/5000 [00:00<00:09, 547.33it/s]












  2%|▏         | 110/5000 [00:00<00:09, 503.72it/s]












  3%|▎         | 168/5000 [00:00<00:09, 533.20it/s]












  4%|▍         | 224/5000 [00:00<00:08, 540.11it/s]












  6%|▌         | 279/5000 [00:00<00:08, 537.84it/s]












  7%|▋         | 337/5000 [00:00<00:08, 549.25it/s]












  8%|▊         | 395/5000 [00:00<00:08, 557.57it/s]












  9%|▉         | 451/5000 [00:00<00:08, 555.63it/s]












 10%|█         | 510/5000 [00:00<00:07, 563.22it/s]












 11%|█▏        | 567/5000 [00:01<00:07, 562.54it/s]












 12%|█▏        | 624/5000 [00:01<00:07, 558.06it/s]












 14%|█▎        | 680/5000 [00:01<00:08, 539.22it/s]












 15%|█▍        | 735/5000 [00:01<00:08, 507.88it/s]












 16%|█▌        | 792/5000 [00:01<00:08, 522.85it/s]












 17%|█▋        | 850/5000 [00:01<00:07,

saved neural_trigram_predictions.npy


Free up RAM.

In [ ]:
# Delete model we don't need.
del neural_trigram_model

### Submission

Upload a submission with the following files to Gradescope:
* Part1.ipynb (rename to match this exactly)
* neural_trigram_predictions.npy
* bigram_predictions.npy

You can upload files individually or as part of a zip file, but if using a zip file be sure you are zipping the files directly and not a folder that contains them.

Be sure to check the output of the autograder after it runs.  It should confirm that no files are missing and that the output files have the correct format.  Note that the test set perplexities shown by the autograder are on a completely different scale from your validation set perplexities due to truncating the distribution and selecting different text.  Don't worry if the values seem much worse.